# <b>spacemake</b> - Quality Control

In [ ]:
import spacemake as smk
from spacemake.config import ConfigFile
from spacemake.project_df import ProjectDF
from spacemake.report import utils as report_utils

from functools import partial

In [ ]:
import os

# limit the resources used by scanpy
os.environ['OMP_NUM_THREADS'] = '1'   # For OpenMP
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # For OpenBLAS
os.environ['MKL_NUM_THREADS'] = '1'  # For MKL (Intel Math Kernel Library)
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'  # For macOS Accelerate
os.environ['NUMEXPR_NUM_THREADS'] = '1'  # For NumExpr

In [ ]:
# Parameters - this cell will be replaced by papermill

run_modes = []
adata_paths = []
split_reads_read_type = "path/to/reads_type_out"
project_id = "project_id"
sample_id = "sample_id"
puck_barcode_file_id_qc = "puck_barcode_file_id"
complete_data_root = "path/to/complete_data"
is_spatial = True
config_yaml_path = "config.yaml"  # at the root spacemake folder
project_df_path = "project_df.csv"  # at the root spacemake folder

In [ ]:
config = ConfigFile.from_yaml(config_yaml_path)
project_df = ProjectDF(project_df_path, config=config)
sample_info = project_df.get_sample_info(project_id, sample_id)

if isinstance(adata_paths, str):
    adata_paths = [adata_paths]

if isinstance(split_reads_read_type, str):
    split_reads_read_type = [split_reads_read_type]

if isinstance(run_modes, str) and run_modes is not None:
    run_modes = [run_modes]
elif run_modes is None:
    run_modes = sample_info["run_mode"]

if (len(run_modes) == len(adata_paths)):
    run_modes_adatas = {
        f'{run_mode}': adata_path for run_mode, adata_path in zip(run_modes, adata_paths)
    }
else:
    raise ValueError("'run_modes' and 'adata_paths' must have the same length")

## QC tables

In [ ]:
sample_info_df = report_utils.create_sample_info_df(project_df, 
                                                    project_id, 
                                                    sample_id, 
                                                    puck_barcode_file_id_qc)
run_modes_df = report_utils.create_run_modes_df(run_modes, project_df)
mapping_stats_df = report_utils.create_mapping_stats_df(
    split_reads_read_type,
    complete_data_root
)
summary_beads_df = report_utils.create_summary_beads_df(
    run_modes_adatas,
)

In [ ]:
visualizer = smk.pl.TabVisualizer()

# Sample Information table
visualizer.add_plot_group(
        smk.pl.PlotGroup(
        name=f"Sample Information",
        description=f"QC tables for the sample",
        plots=[smk.pl.DataFrameTable(
            title="Sample Information",
            description="",
            data=sample_info_df.T.reset_index(names='Variable')
        )]
    )
)

# Run modes table
visualizer.add_plot_group(
        smk.pl.PlotGroup(
        name=f"Run modes",
        description=f"QC tables for the sample",
        plots=[smk.pl.DataFrameTable(
            title="Run modes",
            description="This sample was processed using the following run modes, and run mode variables",
            data=run_modes_df.reset_index(names='Variable')
        )]
    )
)

# Mapping statistics table
visualizer.add_plot_group(
        smk.pl.PlotGroup(
        name=f"Mapping statistics",
        description=f"QC tables for the sample",
        plots=[smk.pl.DataFrameTable(
            title="Mapping statistics",
            description="The mapping statistics are shown here for each method (reads in Millions):",
            data=mapping_stats_df
        )]
    )
)

# Summary Spatial Units table
visualizer.add_plot_group(
        smk.pl.PlotGroup(
        name=f"Summary Spatial Units",
        description=f"QC tables for the sample",
        plots=[smk.pl.DataFrameTable(
            title="Summary Spatial Units",
            description="",
            data=summary_beads_df.reset_index(names='Variable')
        )]
    )
)

In [ ]:
display(visualizer.generate_html())

## QC plots

Each of the QC plots we show on a per run mode basis, to see if there are any downstream differences based on the run mode variable settings.

### Preprocessing

Raw reads were preprocessed according to `adapter_flavor` choice. Reads that ended up below 18nt length after processing were dropped from the analysis.

In [ ]:
from scbamtools.pl import preprocessing_stats, edit_stats
import pandas as pd

In [ ]:
try:
    preprocessing_stats(
        pd.read_csv(f"{complete_data_root}/stats/preprocessing.tsv", sep="\t"),
        sample_id=sample_id,
    )
except (OSError, pd.errors.EmptyDataError):
    print(f"could not find /stats/preprocessing.tsv for {sample_id}. Possibly a merged sample. Skipping preprocessing plots.")

### Barcode matching and error correction

The barcodes observed in this sample were systematically compared to the flowcell tiles. Exact matches were used to select the relevant tiles making up the capture area. Subsequently, all barcodes were matched with up to one substitution, insertion, or deletion. The first match to the capture area barcode universe was kept.
Here are statistics on the match rate and edit frequencies.

Barcodes without a match to the capture area reference were excluded from further analysis, no mapping was attempted for the corresponding reads

In [ ]:
try:
    edit_stats(
        pd.read_csv(f"{complete_data_root}/stats/cb_correct.tsv", sep="\t"),
        query=sample_id,
        ref="capture area",
    )
except (OSError, pd.errors.EmptyDataError):
    print(f"could not find /stats/cb_correct.tsv for {sample_id}. Possibly a merged sample. Skipping barcode match plots.")

### 'Knee'-plot

Below we plot a so called 'Knee'-plot: on the y-axis is the Cummulative sum of reads, on the x-axis are the bead barcodes sorted by number of reads. For single-cell samples, this plot tells you roughly how many spatial units are covered by tissue.

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "Knee-plot"

for run_mode, adata in run_modes_adatas.items():
    plots = []
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Knee-plot for run mode {run_mode}",
        plot_func=partial(smk.pl.knee_plot, adata)
    )
    plots.append(plot)
    
    group = smk.pl.PlotGroup(
        name=f"Run Mode: {run_mode}",
        description=f"Analysis results for {run_mode}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
display(visualizer.generate_html())

### UMI-cutoff plots

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "UMI-cutoff plots"

for run_mode, adata in run_modes_adatas.items():
    plots = []
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Distribution of {metric_desc.lower()} for run mode {run_mode}",
        plot_func=partial(smk.pl.umi_cutoff, adata)
    )
    plots.append(plot)
    
    group = smk.pl.PlotGroup(
        name=f"Run Mode: {run_mode}",
        description=f"Analysis results for {run_mode}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
display(visualizer.generate_html())

### Histogram of metrics over spatial units

Next we show metrics such as number of UMIs, genes, reads and sequencing saturation (PCR bias) per spatial unit. We further distinguish between each run mode, showing one histogram for each.

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "Histogram of metrics over spatial units"

for run_mode, adata in run_modes_adatas.items():
    plots = []
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Distribution of {metric_desc.lower()} for run mode {run_mode}",
        plot_func=partial(smk.pl.plot_histogram_beads, adata)
    )
    plots.append(plot)
    
    group = smk.pl.PlotGroup(
        name=f"Run Mode: {run_mode}",
        description=f"Analysis results for {run_mode}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
display(visualizer.generate_html())

### Nucleotide distribution per spatial unit

Next we bin the data based on reads into quartile. For each run_mode the data is divided into 4 spatial unit, by reads. This means, that the first bin will contain spatial unit which account 25% of the reads, the second will contain spatial unit which account for the second 25% of reads and so on.

<b>For each run mode we plot the nucleotide distribution per quartile.</b>

<b>Only not-meshed run_mode(s) are shown</b>

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "Nucleotide distribution per spatial unit"

for run_mode, adata in run_modes_adatas.items():
    plots = []
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Distribution of {metric_desc.lower()} for run mode {run_mode}",
        plot_func=partial(smk.pl.nucleotide_distribution_per_bead, adata)
    )
    plots.append(plot)
    
    group = smk.pl.PlotGroup(
        name=f"Run Mode: {run_mode}",
        description=f"Analysis results for {run_mode}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
try:
    display(visualizer.generate_html())
except Exception as e:
    print(f"Cannot show these plots with this run mode")

### Shannon entropy and string compression

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "Shannon Entropy and String Compression"

for run_mode, adata in run_modes_adatas.items():
    plots = []
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Distribution of {metric_desc.lower()} for run mode {run_mode}",
        plot_func=partial(smk.pl.entropy_compression, adata)
    )
    plots.append(plot)
    
    group = smk.pl.PlotGroup(
        name=f"Run Mode: {run_mode}",
        description=f"Analysis results for {run_mode}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
display(visualizer.generate_html())

In [ ]:
if not is_spatial:
    print("WARNING: This is not a spatial sample. Some analyses will be skipped.")
    # Define a variable to track whether to continue with spatial-specific code
    continue_spatial_analyses = False
else:
    continue_spatial_analyses = True

### Spatial QC

In [ ]:
spatial_metrics = {
    "n_genes_by_counts": (
        "Number of genes, per spatial unit",
        "Total number of distinct genes detected in each spatial unit, reflecting transcriptional diversity and capture efficiency."
    ),
    "total_counts": (
        "Number of UMIs, per spatial unit",
        "Sum of unique molecular identifiers (UMIs) observed per spatial unit; a direct proxy for transcript abundance and sequencing depth."
    ),
    "pct_counts_mt": (
        "Percentage of mitochondrial counts, per spatial unit",
        "Fraction of UMIs mapping to mitochondrial genes in each spatial unit - elevated levels may indicate lower data quality."
    ),
    "n_reads": (
        "Number of reads, per spatial unit",
        "Total raw sequencing reads assigned to each spatial unit before UMI collapsing, providing an overview of raw depth."
    ),
    "reads_per_counts": (
        "reads/UMI, per spatial unit",
        "Average number of reads supporting each UMI in a spatial unit, a measure of PCR duplication bias: values close to 1 imply high library complexity and efficient sequencing, whereas higher ratios suggest saturation and redundant sequencing."
    ),
    "n_joined": (
        "Number of barcoded capture spots, per spatial unit",
        "Count of individual barcoded capture spots merged to form the given spatial unit (relevant for meshed or aggregated run modes)."
    ),
    "exact_entropy": (
        "Shannon entropy, per spatial unit",
        "Shannon entropy calculated from bead barcode sequences within each spatial unit; higher entropy denotes more uniform barcode usage (greater diversity)."
    ),
    "exact_compression": (
        "Barcode length after compression, per spatial unit",
        "Compressed length of the concatenated barcode string for each spatial unit; shorter lengths indicate higher redundancy and lower information content."
    ),
}

In [ ]:
if continue_spatial_analyses:
    visualizer = smk.pl.TabVisualizer()

    for run_mode, adata in run_modes_adatas.items():
        plots = []

        for metric_key, metric_desc in spatial_metrics.items():
            plot = smk.pl.Plot(
                title=metric_desc[0],
                description=metric_desc[1],
                plot_func=partial(smk.pl.spatial, adata, color=metric_key)
            )
            plots.append(plot)
        
        group = smk.pl.PlotGroup(
            name=f"Run Mode: {run_mode}",
            description=f"Analysis results for {run_mode}",
            plots=plots
        )
        
        visualizer.add_plot_group(group)

In [ ]:
if continue_spatial_analyses:
    try:
        display(visualizer.generate_html())
    except KeyError as e:
        print(f"Could not plot because of {e}")